1. Загрузка признаков для модели в базу данных
Во время работы сервиса ему необходимо для каждого user_id предсказывать топ-5 постов, которые с наибольшей вероятностью посмотрит человек. Для того, чтобы это сделать модели необходимы заранее загрузить посчитанные признаки для обученной модели из предыдущего степа. Лучше всего их посчитать локально на JupyterHub и с помощью метода pd.to_sql записать в базу данных. 

In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [2]:
df = pd.read_csv('user_features.csv') # создаем датафрейм с посчитанными признаками

df.to_sql('user_features_07122301', con=engine) # записываем таблицу

727

2. Считывание признаков из базы данных
Загрузка признаков представляет собой чтение таблицы из базы данных с помощью SQL в память. Однако, Python и его библиотеки полны неожиданного поведения. Например, когда мы работаем с функцией pd.read_sql , то данная функциональность потребляет в 4 раза больше памяти, чем необходимо для хранения данных. Подробнее можно почитать об этом тут. Решением такой проблемы является загрузка данных по кусочкам (или иначе говоря чанкам).
В данном задании вам необходимо написать код, который бы записывал признаки в базу данных. После чего написать функцию def load_features() -> pd.DataFrame , которая бы загружала признаки с помощью функции batch_load_sql, которую мы предоставили вам выше. 

In [3]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [4]:
def load_features() -> pd.DataFrame:
    query = 'SELECT * FROM user_features_07122301'
    user_features = batch_load_sql(query)
    return user_features

In [5]:
user_features = load_features()
user_features.head()

,index,Unnamed: 0,user_id,post_id,target,gender,age,country,city,exp_group,os,source,user_rate,text,topic,post_rate,timestamp
0,0,0,29863,1269,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Blair and Brown criticised by MPs\n\nLabour MP...,politics,0.103448,2021-10-01 21:54:51
1,1,1,29863,1297,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Could rivalry overshadow election?\n\nTony Bla...,politics,0.095238,2021-10-02 14:07:02
2,2,2,29863,1524,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Italy aim to rattle England\n\nItaly coach Joh...,sport,0.142857,2021-11-17 12:30:06
3,3,3,29863,1601,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Isinbayeva claims new world best\n\nPole vault...,sport,0.000000,2021-10-01 20:27:04
4,4,4,29863,1806,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Campbell rescues Arsenal\n\nSol Campbell prove...,sport,0.157895,2021-10-02 11:03:11
